In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
# Check for GPU
import tensorflow as tf
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")
else:
    print(tf.config.list_physical_devices())
    print('\nCUDA GPU: ' + str(tf.test.is_gpu_available(cuda_only=True)))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

CUDA GPU: False


# Data Preprocessing

In [3]:
import os
os.chdir('..')

df = pd.read_csv('./hourly02-ithaca/hourly02-NY_Ithaca_13_E.csv', header = 0, index_col = 0)

In [4]:
Date = pd.to_datetime(df.UTC_DATE, format='%Y%m%d', errors='coerce')
+ pd.to_timedelta(df.UTC_TIME//100, unit = 'hours')
df['Time'] = Date

In [5]:
df.columns

Index(['WBANNO', 'UTC_DATE', 'UTC_TIME', 'LST_DATE', 'LST_TIME', 'CRX_VN',
       'LONGITUDE', 'LATITUDE', 'T_CALC', 'T_HR_AVG', 'T_MAX', 'T_MIN',
       'P_CALC', 'SOLARAD', 'SOLARAD_FLAG', 'SOLARAD_MAX', 'SOLARAD_MAX_FLAG',
       'SOLARAD_MIN', 'SOLARAD_MIN_FLAG', 'SUR_TEMP_TYPE', 'SUR_TEMP',
       'SUR_TEMP_FLAG', 'SUR_TEMP_MAX', 'SUR_TEMP_MAX_FLAG', 'SUR_TEMP_MIN',
       'SUR_TEMP_MIN_FLAG', 'RH_HR_AVG', 'RH_HR_AVG_FLAG', 'SOIL_MOISTURE_5',
       'SOIL_MOISTURE_10', 'SOIL_MOISTURE_20', 'SOIL_MOISTURE_50',
       'SOIL_MOISTURE_100', 'SOIL_TEMP_5', 'SOIL_TEMP_10', 'SOIL_TEMP_20',
       'SOIL_TEMP_50', 'SOIL_TEMP_100', 'Time'],
      dtype='object')

In [6]:
data = df[['T_CALC', 'T_HR_AVG', 'T_MAX', 'T_MIN',
       'P_CALC', 'SOLARAD', 'SOLARAD_MAX',
       'SOLARAD_MIN', 'SUR_TEMP',
           'SUR_TEMP_MAX', 'SUR_TEMP_MIN', 'RH_HR_AVG']]

In [7]:
data.index = df['Time']

In [8]:
# check for N/A
data.min()

T_CALC         -30.8
T_HR_AVG       -29.3
T_MAX          -28.4
T_MIN          -30.9
P_CALC           0.0
SOLARAD          0.0
SOLARAD_MAX      0.0
SOLARAD_MIN      0.0
SUR_TEMP       -35.8
SUR_TEMP_MAX   -61.0
SUR_TEMP_MIN   -36.0
RH_HR_AVG        0.0
dtype: float64

In [9]:
data

,T_CALC,T_HR_AVG,T_MAX,T_MIN,P_CALC,SOLARAD,SOLARAD_MAX,SOLARAD_MIN,SUR_TEMP,SUR_TEMP_MAX,SUR_TEMP_MIN,RH_HR_AVG
Time,,,,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,8.8,NaN,NaN,0.0
2004-10-27,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6.7,NaN,NaN,0.0
2004-10-28,7.8,7.6,8.0,7.3,0.0,0.0,NaN,NaN,6.1,NaN,NaN,0.0
2004-10-28,6.5,7.0,7.8,6.5,0.0,0.0,NaN,NaN,5.6,NaN,NaN,0.0
2004-10-28,5.4,6.2,6.5,5.4,0.0,0.0,NaN,NaN,5.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06,1.1,1.3,2.1,0.3,0.0,0.0,0.0,0.0,-1.6,-0.9,-2.0,76.0
2023-11-06,-0.1,0.0,1.0,-1.1,0.0,0.0,0.0,0.0,-2.3,-1.9,-2.8,80.0
2023-11-06,-0.5,-0.4,0.1,-0.7,0.0,0.0,0.0,0.0,-3.1,-2.8,-3.3,83.0


In [10]:
data.isna().sum()

T_CALC           1384
T_HR_AVG         1446
T_MAX            1385
T_MIN            1389
P_CALC            832
SOLARAD           570
SOLARAD_MAX      9757
SOLARAD_MIN      9757
SUR_TEMP          724
SUR_TEMP_MAX     9911
SUR_TEMP_MIN     9911
RH_HR_AVG       48248
dtype: int64

In [11]:
# Check data types
data.dtypes

T_CALC          float64
T_HR_AVG        float64
T_MAX           float64
T_MIN           float64
P_CALC          float64
SOLARAD         float64
SOLARAD_MAX     float64
SOLARAD_MIN     float64
SUR_TEMP        float64
SUR_TEMP_MAX    float64
SUR_TEMP_MIN    float64
RH_HR_AVG       float64
dtype: object

In [12]:
data.shape

(166759, 12)

In [13]:
# forward fill the missing values  
data.ffill(axis = 0, inplace = True) 

In [14]:
data

,T_CALC,T_HR_AVG,T_MAX,T_MIN,P_CALC,SOLARAD,SOLARAD_MAX,SOLARAD_MIN,SUR_TEMP,SUR_TEMP_MAX,SUR_TEMP_MIN,RH_HR_AVG
Time,,,,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,8.8,NaN,NaN,0.0
2004-10-27,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6.7,NaN,NaN,0.0
2004-10-28,7.8,7.6,8.0,7.3,0.0,0.0,NaN,NaN,6.1,NaN,NaN,0.0
2004-10-28,6.5,7.0,7.8,6.5,0.0,0.0,NaN,NaN,5.6,NaN,NaN,0.0
2004-10-28,5.4,6.2,6.5,5.4,0.0,0.0,NaN,NaN,5.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06,1.1,1.3,2.1,0.3,0.0,0.0,0.0,0.0,-1.6,-0.9,-2.0,76.0
2023-11-06,-0.1,0.0,1.0,-1.1,0.0,0.0,0.0,0.0,-2.3,-1.9,-2.8,80.0
2023-11-06,-0.5,-0.4,0.1,-0.7,0.0,0.0,0.0,0.0,-3.1,-2.8,-3.3,83.0


In [15]:
# drop NaN at the top
data.dropna(inplace = True)

In [16]:
# set target
data['target'] = data['T_HR_AVG']

In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle = False)

In [18]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

with open('./LSTM/models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [19]:
train.shape

(126057, 13)

In [20]:
test.shape

(31515, 13)

In [21]:
# splitting data into sequences
def split_sequences(features, target, seq_len, forecast_len):
    X,y = list(), list()
    for i in range(len(features)):
        end_input = i + seq_len
        end_predict = end_input + forecast_len
        if end_predict > len(features)-1:
            break
        seq_x, seq_y = features[i:end_input,:], target[end_input:end_predict]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float64), tf.convert_to_tensor(y, dtype=tf.float64)

# Define Model

In [22]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RNN, LSTMCell, Input, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

class MyModel(tf.keras.Model):

    def __init__(self, input_shape, output_shape, name = 'LSTM'):
        super().__init__(name = name)
        self.input_layer = Input(shape = input_shape, name = 'input')
        self.lstm1 = LSTM(units=30, activation = 'tanh', input_shape = input_shape, return_sequences=False, name = 'lstm_1')
        self.dense1 = Dense(units=output_shape, activation = 'relu', name = 'dense_1')
        #self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dense1(x)
        #if training:
        #  x = self.dropout(x, training=training)
        return x
    
    def summary(self):
        model = Model(inputs = [self.input_layer], outputs = self.call(self.input_layer), name = self.name)
        return model.summary()

# Model Training
## input length : output length = 16:4

In [23]:
# prepare sequences
seq_len = 16
forecast_len = 4
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [24]:
X_train.shape

TensorShape([126037, 16, 12])

In [25]:
y_train.shape

TensorShape([126037, 4])

In [26]:
# create model instance
model_name = 'LSTM_16-4'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM_16-4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 16, 12)]          0         
                                                                 
 lstm_1 (LSTM)               (None, 30)                5160      
                                                                 
 dense_1 (Dense)             (None, 4)                 124       
                                                                 
Total params: 5,284
Trainable params: 5,284
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 6s 4ms/step - loss: 0.0027 - mse: 0.0027 - acc: 0.5396 - val_loss: 8.3953e-04 - val_mse: 8.3953e-04 - val_acc: 0.5805
Epoch 2/30
1135/1135 [==============================] - 4s 4ms/step - loss: 7.3273e-04 - mse: 7.3273e-04 - acc: 0.5635 - val_loss: 6.3779e-04 - val_mse: 6.3779e-04 - val_acc: 0.6427
Epoch 3/30
1135/1135 [==============================] - 4s 4ms/step - loss: 6.2521e-04 - mse: 6.2521e-04 - acc: 0.5816 - val_loss: 5.4084e-04 - val_mse: 5.4084e-04 - val_acc: 0.5614
Epoch 4/30
1135/1135 [==============================] - 4s 4ms/step - loss: 5.7789e-04 - mse: 5.7789e-04 - acc: 0.6144 - val_loss: 5.2769e-04 - val_mse: 5.2769e-04 - val_acc: 0.5684
Epoch 5/30
1135/1135 [==============================] - 4s 4ms/step - loss: 5.4966e-04 - mse: 5.4966e-04 - acc: 0.6177 - val_loss: 4.9048e-04 - val_mse: 4.9048e-04 - val_acc: 0.4436
Epoch 6/30
1135/1135 [==============================] - 5s 4ms/step - loss: 5.3416e-04 - mse: 5.34

In [28]:
# save trained model
model.save('./LSTM/models/' + model_name)
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_16-4\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_16-4\assets


985/985 [==============================] - 1s 910us/step


In [29]:
from sklearn.metrics import mean_squared_error
print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.000410448137396148
test set: 0.0004588499961577377


## input length : output length = 24:6

In [30]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 24
forecast_len = 6
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [31]:
# create model instance
model_name = 'LSTM_24-6'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM_24-6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 24, 12)]          0         
                                                                 
 lstm_1 (LSTM)               (None, 30)                5160      
                                                                 
 dense_1 (Dense)             (None, 6)                 186       
                                                                 
Total params: 5,346
Trainable params: 5,346
Non-trainable params: 0
_________________________________________________________________


In [32]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 8s 6ms/step - loss: 0.0025 - mse: 0.0025 - acc: 0.4657 - val_loss: 9.8442e-04 - val_mse: 9.8442e-04 - val_acc: 0.4927
Epoch 2/30
1135/1135 [==============================] - 6s 5ms/step - loss: 9.4125e-04 - mse: 9.4125e-04 - acc: 0.5360 - val_loss: 9.0479e-04 - val_mse: 9.0479e-04 - val_acc: 0.3894
Epoch 3/30
1135/1135 [==============================] - 6s 5ms/step - loss: 8.4836e-04 - mse: 8.4836e-04 - acc: 0.5586 - val_loss: 0.0012 - val_mse: 0.0012 - val_acc: 0.5916
Epoch 4/30
1135/1135 [==============================] - 6s 5ms/step - loss: 8.2777e-04 - mse: 8.2777e-04 - acc: 0.5712 - val_loss: 7.3444e-04 - val_mse: 7.3444e-04 - val_acc: 0.5192
Epoch 5/30
1135/1135 [==============================] - 6s 5ms/step - loss: 8.0340e-04 - mse: 8.0340e-04 - acc: 0.5757 - val_loss: 7.1050e-04 - val_mse: 7.1050e-04 - val_acc: 0.5575
Epoch 6/30
1135/1135 [==============================] - 6s 5ms/step - loss: 7.8723e-04 - mse: 7.8723e-04 -

In [33]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_24-6\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_24-6\assets


984/984 [==============================] - 1s 1ms/step
mean_squared_error
train set: 0.0006072851833836482
test set: 0.0006667094617247199


## input length : output length = 32:8

In [34]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 32
forecast_len = 8
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [35]:
# create model instance
model_name = 'LSTM_32-8'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM_32-8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 32, 12)]          0         
                                                                 
 lstm_1 (LSTM)               (None, 30)                5160      
                                                                 
 dense_1 (Dense)             (None, 8)                 248       
                                                                 
Total params: 5,408
Trainable params: 5,408
Non-trainable params: 0
_________________________________________________________________


In [36]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

Epoch 1/30
1135/1135 [==============================] - 9s 7ms/step - loss: 0.0115 - mse: 0.0115 - acc: 0.3907 - val_loss: 0.0015 - val_mse: 0.0015 - val_acc: 0.3954
Epoch 2/30
1135/1135 [==============================] - 7s 6ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.4960 - val_loss: 0.0012 - val_mse: 0.0012 - val_acc: 0.4857
Epoch 3/30
1135/1135 [==============================] - 7s 7ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5063 - val_loss: 0.0013 - val_mse: 0.0013 - val_acc: 0.4983
Epoch 4/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5228 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.4558
Epoch 5/30
1135/1135 [==============================] - 7s 6ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5332 - val_loss: 0.0012 - val_mse: 0.0012 - val_acc: 0.5103
Epoch 6/30
1135/1135 [==============================] - 7s 6ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5434 - val_loss: 9.7765e-04 - val_mse: 9.7765e-04 - val_acc: 0.4

In [37]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_32-8\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_32-8\assets


984/984 [==============================] - 1s 1ms/step
mean_squared_error
train set: 0.001011904090414483
test set: 0.0010585031586038249


## input length : output length = 40:10

In [38]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 40
forecast_len = 10
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [41]:
# create model instance
model_name = 'LSTM_40-10'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM_40-10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 40, 12)]          0         
                                                                 
 lstm_1 (LSTM)               (None, 30)                5160      
                                                                 
 dense_1 (Dense)             (None, 10)                310       
                                                                 
Total params: 5,470
Trainable params: 5,470
Non-trainable params: 0
_________________________________________________________________


In [42]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)


Epoch 1/30
1135/1135 [==============================] - 11s 9ms/step - loss: 0.0051 - mse: 0.0051 - acc: 0.3866 - val_loss: 0.0022 - val_mse: 0.0022 - val_acc: 0.4507
Epoch 2/30
1135/1135 [==============================] - 10s 9ms/step - loss: 0.0016 - mse: 0.0016 - acc: 0.4590 - val_loss: 0.0016 - val_mse: 0.0016 - val_acc: 0.3866
Epoch 3/30
1135/1135 [==============================] - 10s 9ms/step - loss: 0.0015 - mse: 0.0015 - acc: 0.4812 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.5158
Epoch 4/30
1135/1135 [==============================] - 9s 8ms/step - loss: 0.0014 - mse: 0.0014 - acc: 0.4917 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.4284
Epoch 5/30
1135/1135 [==============================] - 10s 8ms/step - loss: 0.0014 - mse: 0.0014 - acc: 0.4998 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.4013
Epoch 6/30
1135/1135 [==============================] - 10s 8ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.5082 - val_loss: 0.0017 - val_mse: 0.0017 - val_acc: 0.5316

In [43]:
# save trained model
model.save('./LSTM/models/' + model_name)
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_40-10\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM_40-10\assets


984/984 [==============================] - 2s 2ms/step
mean_squared_error
train set: 0.0013483855286335529
test set: 0.00137299509520843
